# 02. 기본적인 추천 시스템

<br>

## 02-01. 데이터 로드

In [2]:
import numpy as np
import pandas as pd
import os
import warnings

In [3]:
warnings.filterwarnings('ignore')

<br>

- 사용자 정보 데이터

In [4]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')
users = users.set_index('user_id')
users.head()

,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


<br>

- 제품 정보 데이터

In [5]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.set_index('movie_id')
movies.head()

,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,Children's,Comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


<br>

- 평점 정보 데이터

In [6]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.set_index('user_id')
ratings.head()

,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


<br>

## 02-02. 인기제품 방식
- **개별 사용자에 대한 정보가 없는 경우나, 정확도에 관계없이 가장 간단한 추천을 제공해야 하는 상황에서**
    
    **사용할 수 있는 방법은 모든 사람에게 똑같은 추천을 하는 것**
- 모든 사람들에게 가장 인기 있는 제품 (best-seller)을 추천하는 것이 가장 합리적
- 각 제품에 대한 평가를 평균해서 평균값이 가장 높은 것을 순서대로 추천

In [7]:
def recom_movie1(n_items=5):
    movie_sort = movie_mean.sort_values(ascending=False)[:n_items]
    recom_movies = movies.loc[movie_sort.index]
    recommendations = recom_movies['title']
    return recommendations

In [8]:
movie_mean = ratings.groupby(['movie_id'])['rating'].mean()
recom_movie1(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

In [9]:
def recom_movie2(n_items):
   return movies.loc[movie_mean.sort_values(ascending=False)[:n_items].index]['title']

In [10]:
recom_movie2(5)

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object

<br>

## 02-03. 추천 시스템의 정확도 측정
- 추천 시스템의 성능은 '정확성'으로 표시
- RMSE

$$RMSE = \sqrt{\frac{1}{N}\sum^N (y_i-\hat{y_i})^2}$$

In [11]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

- best-seller 방식으로 구한 예측값의 RMSE를 계산
    - 사용자가 평가한 평점평균을 `y_pred` (평점평균이 해당 영화에 대한 예측값)

In [12]:
rmse = []
for user in set(ratings.index):
    y_true = ratings.loc[user]['rating']
    y_pred = movie_mean[ratings.loc[user]['movie_id']]
    accuracy = RMSE(y_true, y_pred)
    rmse.append(accuracy)
    
print(np.mean(rmse))

0.996007224010567


<br>

## 02-04. 사용자 집단별 추천
- **비슷한 특성의 사람들을 묶은 소집단으로 만든 다음, 각 집단의 평점평균을 바탕으로 추천하는 방식**

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('u.user', sep='|', names=u_cols, encoding='latin-1')

In [15]:
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 
          'unknown', 'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 
          'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
movies = pd.read_csv('u.item', sep='|', names=i_cols, encoding='latin-1')
movies = movies.filter(['movie_id', 'title'], axis=1)

In [16]:
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('u.data', sep='\t', names=r_cols, encoding='latin-1') 
ratings = ratings.drop('timestamp', axis=1)

<br>

- 훈련, 테스트 데이터 분리

In [17]:
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

- RMSE 계산 함수 정의

In [18]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

- 모델별 RMSE 계산 함수 정의

In [19]:
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

<br>

#### Full Matrix
<br>

- 기본적으로 예측은 (한 사용자 - 한 영화) 조합에 대해서 이루어짐
- 사용자의 특성에 따라 집단을 나누어서 예측 하려면 평점 데이터뿐만 아니라, 특성 (성별, 나이, 직업 등)에 대한 정보가 담긴 사용자 데이터가 필요
    
    $\rightarrow$ 평점 데이터와 사용자 데이터를 평합
    
    $\rightarrow$ 평점 데이터의 Full Matrix 계산
    
<br>

- **Full Matrix : (사용자-영화-평점)으로 이루어져 있는 데이터를,**

    **각 사용자와 영화를 각 차원으로 하는 전체 배열**

<br>

- train 데이터로 Full Matrix 계산

In [20]:
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [21]:
rating_matrix

movie_id,1,2,3,4,5,6,7,8,9,10,...,1666,1668,1670,1672,1673,1675,1678,1679,1680,1681
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

- 전체 평균으로 예측치를 계산하는 기본 모델
    - 해당 영화가 평균 데이터에 존재하지 않으면 3.0을 반환

In [22]:
def best_seller(user_id, movie_id):
    try:
        rating = train_mean[movie_id]
    except:
        rating = 3.0
    return rating

In [23]:
train_mean = x_train.groupby(['movie_id'])['rating'].mean()

In [24]:
train_mean

movie_id
1       3.895522
2       3.173913
3       3.058824
4       3.575949
5       3.291667
          ...   
1675    3.000000
1678    1.000000
1679    3.000000
1680    2.000000
1681    3.000000
Name: rating, Length: 1641, dtype: float64

In [25]:
score(best_seller)

1.0256144290860054

- Full Matrix를 사용자 데이터와 병합

In [26]:
merged_ratings = pd.merge(x_train, users)
users = users.set_index('user_id')

- gender별 평점평균 계산

In [27]:
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

<br>

#### Gender 기준 추천
- gender별 평균을 예측

In [ ]:
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:
        gender = users.loc[user_id]['sex']
        if gender in g_mean[movie_id]:
            gender_rating = g_mean[movie_id][gender]
        else:
            gender_rating = 3.0
    else:
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

<br>

## 02.05. 내용 기반 필터링
- **내용 기반 필터링 (Content-Based Filtering : CB)는 아이템의 내용을 분석해서, 아이템간 유사도를 계산하고 이를 바탕으로 추천**

<br>

#### 내용 기반 필터링 절차
1. 각 아이템간의 유사도를 계산
2. 추천 대상이 되는 사용자가 선호하는 아이템을 선정
3. 선정된 아이템과 가장 유사도가 높은 N개의 아이템을 탐색
- N의 크기를 어떻게 선정할 것인가에 대한 이슈 존재
4. N개의 아이템을 사용자에게 추천

<br>

- **아이템의 내용 중에서 텍스트를 분석하는 경우에는, 아이템간의 유사도를 측정하는 지표로 TF-IDF를 주로 사용**
    - TF-IDF를 사용하려면, 어떤 단어가 해당 문서에 얼마나 자주 등장하는 가 하는 것과 (item frequdncy)
    
        다른 문서에 비해서 상대적으로 얼마나 더 자주 등장하는가 (inverse documentary frequency)를 계산
        
        $\rightarrow$ tf와 idf를 조합해서 각 문서(아이템)에 등장하는 모든 단어의 가중치(중요도)를 계산
        
        $\rightarrow$ 이들 단어의 가중치가 문서 간에 얼마나 유사한지를 cosine similarity 지표를 사용해서 계싼
        
        $\rightarrow$ 단어의 단순한 빈도뿐만 아니라, 다른 무서와 비교한 상대적 빈도도 고려하기 때문에 더 정확한 분석 가능
    
<br>

#### 영화별 메타 데이터 로드

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

In [2]:
movies = pd.read_csv('movies_metadata.csv', encoding='latin-1', low_memory=False)
movies = movies[['id', 'title', 'overview']]
len(movies)

45442

In [3]:
movies.head()

,id,title,overview
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...
2,15602,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,31357,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,11862,Father of the Bride Part II,Just when George Banks has recovered from his ...


- 전처리 (중복제거, 결측값 제거)

In [4]:
movies = movies.drop_duplicates()
movies = movies.dropna()
movies['overview'] = movies['overview'].fillna('')
len(movies)

44300

- 불용어 제거 후 tf-idf 계산

In [5]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['overview'])

- tf-idf 값을 통하여, 영화간 유사도 계산

In [6]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim = pd.DataFrame(cosine_sim, index=movies.index, columns=movies.index)

In [7]:
indices = pd.Series(movies.index, index=movies['title'])

- 영화 제목을 받아서 추천 영화를 (유사한 영화)를 반환

In [8]:
def content_recommender(title, n_of_recomm):

    idx = indices[title]
    sim_scores = cosine_sim[idx]
    sim_scores = sim_scores.sort_values(ascending=False)[1:n_of_recomm+1]
    return movies.loc[sim_scores.index]['title']

In [9]:
print(content_recommender('The Lion King', 5))

34664    How the Lion Cub and the Turtle Sang a Song
9339                               The Lion King 1Â½
9101                  The Lion King 2: Simba's Pride
42806                                           Prey
25637                                 Fearless Fagan
Name: title, dtype: object


In [10]:
print(content_recommender('The Dark Knight Rises', 10))

12468                                      The Dark Knight
149                                         Batman Forever
1321                                        Batman Returns
15497                           Batman: Under the Red Hood
584                                                 Batman
21179    Batman Unmasked: The Psychology of the Dark Kn...
9216                    Batman Beyond: Return of the Joker
18021                                     Batman: Year One
19778              Batman: The Dark Knight Returns, Part 1
3085                          Batman: Mask of the Phantasm
Name: title, dtype: object
